In [ ]:
# join_silver_claims_enriched.ipynb
# SOURCE: Snapshot join of current claims and current provider attributes
# OUTPUT: kardia_silver.silver_claims_enriched (for Gold KPIs)
# PATTERN: LEFT JOIN from claims to SCD‑2 providers, keeping only the current row
# TRIGGER: CTAS; rerun after any update to claims or providers

# NOTE:
# - LEFT JOIN preserves all claims
# - IS_CURRENT = TRUE ensures only the current provider row is used

# Optional library bootstrap for ephemeral jobs clusters
%run ../../99_utilities/bootstrap_kflow

from kflow.auth_adls import ensure_adls_oauth

# Configure Spark with ADLS OAuth credentials and return base ABFS path
abfss_base = ensure_adls_oauth()

In [0]:
%sql
-- Ensure Silver DB exists
USE CATALOG hive_metastore;
CREATE DATABASE IF NOT EXISTS kardia_silver;
USE kardia_silver;

In [ ]:
%sql
-- 2. Join claims to current provider attributes (no CTE – inline subquery)
CREATE OR REPLACE TABLE silver_claims_enriched AS
WITH current_providers AS (
  SELECT provider_id,
         provider_specialty,
         provider_location
  FROM   kardia_silver.silver_providers
  WHERE  is_current = TRUE
)
SELECT
    c.claim_id,
    c.patient_id,
    c.provider_id,
    c.claim_amount,
    c.claim_date,
    c.diagnosis_code,
    c.procedure_code,
    c.claim_status,
    c.claim_type,
    c.claim_submission_method,
    c._ingest_ts,
    c._batch_id,
    c._source_file,
    cp.provider_specialty,
    cp.provider_location
FROM kardia_silver.silver_claims c
LEFT JOIN current_providers cp
  ON c.provider_id = cp.provider_id;

In [0]:
%sql
-- 3. Preview: Top 5 enriched claims
SELECT *
FROM kardia_silver.silver_claims_enriched
ORDER BY claim_date DESC
LIMIT 5;